# Wrappers example

In this notebook, we will explore *Sinergym*'s pre-defined wrappers and how to use them.

You can also create your own wrappers by inheriting from *gym.Wrapper* or any of its variants.

In [4]:
import gymnasium as gym
import numpy as np

import sinergym
from sinergym.utils.wrappers import *

## Multi-objective wrapper

[MO-Gymnasium](https://github.com/Farama-Foundation/MO-Gymnasium) is an open-source Python library for developing and comparing multi-objective reinforcement learning algorithms. 

Available MO-Gymnasium environments return a reward vector instead of a scalar value, one for each objective.

This wrapper enables *Sinergym* to return a reward vector. This way, *Sinergym* is made compatible with both multi-objective algorithms and algorithms that work with a traditional reward value.

We can transform the returned reward into a vector using as follows:

In [2]:
env = gym.make('Eplus-5zone-hot-discrete-v1')
env = MultiObjectiveReward(env, reward_terms=['energy_term', 'comfort_term'])

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-discrete-v1
#==============================================================================================#
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1
[MODEL] (INFO) : Model Config is correct.
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names.
[MODEL] (INFO) : Updated building model Output:Meter with meter names.
[MODEL] (INFO) : Runperiod established.
[MODEL] (INFO) : Episode length (seconds): 31536000.0
[MODEL] (INFO) : timestep size (seconds): 900.0
[MODEL] (INFO) : timesteps per episode: 35040
[REWARD] (INFO) : Reward function initialized.
[ENVIRONMENT] (INFO) : Environment created successfully.
[WRAPPER DiscretizeEnv] (INFO) : New Discrete Space and mapping: Discrete(10)
[WRAPPER Discretize

Make sure that `reward_terms` are available in the `info` dict returned by the environment's `step` method. Otherwise, an execution error will occur.

By default, *Sinergym* environments return all reward terms of the reward class in the `info` dict.

In [3]:
env.reset()
action = env.action_space.sample()
obs, reward, terminated, truncated, info = env.step(action)
env.close()

print(reward)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode.
[ENVIRONMENT] (INFO) : Episode 1: Eplus-5zone-hot-discrete-v1
#----------------------------------------------------------------------------------------------#
[MODEL] (INFO) : Episode directory created.
[MODEL] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODEL] (INFO) : Adapting weather to building model.
[ENVIRONMENT] (INFO) : Saving episode output path in /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1/episode-1/output.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
[ENVIRONMENT] (INFO) : Episode 1 started.
[ENVIRONMENT] (INFO) : Environment closed. [Eplus-5zone-hot-discrete-v1]                  
Simulation Progress [Episode 1]: 100%|██████████| 100/100 [00:00<00:00, 4527.09%/s, 100% completed][-0.005897377870609513, -0.09451834

## Previous observation wrappers

This wrapper will add previous timestep observation values to the current environment observation. 

You can select the variables whose previous observed values should be tracked. The observation space will be updated with the corresponding new dimension.

In [4]:
env = gym.make('Eplus-5zone-hot-discrete-v1')
env = PreviousObservationWrapper(env, previous_variables=[
    'htg_setpoint',
    'clg_setpoint',
    'air_temperature'])

#==============================================================================================#   
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.                               
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-discrete-v1                             
#==============================================================================================# 
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1
[MODEL] (INFO) : Model Config is correct.                                            
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names. 
[MODEL] (INFO) : Updated building model Output:Meter with meter names.               
[MODEL] (INFO) : Runperiod established.                                              
[MODEL] (INFO) : Episode length (seconds): 31536000.0                                
[MODEL] (INFO) : timestep size (seconds): 900.0                                      
[MODEL]

You can see how the observation values have been updated:

In [ ]:
env.reset()
obs, _, _, _, _ = env.step(env.action_space.sample())
obs_dict = env.get_obs_dict(obs)
env.close()

print('NEW OBSERVATION: ', obs_dict)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode.
[ENVIRONMENT] (INFO) : Episode 1: Eplus-5zone-hot-discrete-v1
#----------------------------------------------------------------------------------------------#
[MODEL] (INFO) : Episode directory created.
[MODEL] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODEL] (INFO) : Adapting weather to building model.
[ENVIRONMENT] (INFO) : Saving episode output path in /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1/episode-1/output.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
[ENVIRONMENT] (INFO) : Episode 1 started.
[ENVIRONMENT] (INFO) : Environment closed. [Eplus-5zone-hot-discrete-v1]                  
Simulation Progress [Episode 1]: 100%|██████████| 100/100 [00:00<00:00, 9127.97%/s, 100% completed]NEW OBSERVATION:  {'month': np.floa

## Datetime wrapper

This wrapper will replace the `day` value with the `is_weekend` flag, and `hour` and `month` with codified *sin* and *cos* values.

The observation space is also automatically updated.

In [6]:
env = gym.make('Eplus-5zone-hot-discrete-v1')
env = DatetimeWrapper(env)

#==============================================================================================#   
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.                               
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-discrete-v1                             
#==============================================================================================# 
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res2
[MODEL] (INFO) : Model Config is correct.                                            
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names. 
[MODEL] (INFO) : Updated building model Output:Meter with meter names.               
[MODEL] (INFO) : Runperiod established.                                              
[MODEL] (INFO) : Episode length (seconds): 31536000.0                                
[MODEL] (INFO) : timestep size (seconds): 900.0                                      
[MODEL]

This wrapper removes the observation variables `month`, `day`, and `hour`, and replace them by `month_sin`, `month_cos`, `is_weekend`, `hour_sin`, and `hour_cos`:

In [ ]:
env.reset()
obs, _, _, _, _ = env.step(env.action_space.sample())
obs_dict = env.get_obs_dict(obs)
env.close()
print('NEW OBSERVATION: ', obs_dict)

print('NEW OBSERVATION: ', obs_dict)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode.
[ENVIRONMENT] (INFO) : Episode 1: Eplus-5zone-hot-discrete-v1
#----------------------------------------------------------------------------------------------#
[MODEL] (INFO) : Episode directory created.
[MODEL] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODEL] (INFO) : Adapting weather to building model.
[ENVIRONMENT] (INFO) : Saving episode output path in /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res2/episode-1/output.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
[ENVIRONMENT] (INFO) : Episode 1 started.
[ENVIRONMENT] (INFO) : Environment closed. [Eplus-5zone-hot-discrete-v1]                  
Simulation Progress [Episode 1]: 100%|██████████| 100/100 [00:00<00:00, 6941.45%/s, 100% completed]NEW OBSERVATION:  {'month_cos': np.

## Action normalization wrapper

Here's an example of how to normalize a continuous action space using the `NormalizeAction` wrapper.

If the normalization range is not defined, it will be `[-1,1]` by default.

In [8]:
# Create a continuous environment
env = gym.make('Eplus-5zone-hot-continuous-v1')
print('ORIGINAL ACTION SPACE: ', env.get_wrapper_attr('action_space'))

# Apply the normalization wrapper
env = NormalizeAction(env, normalize_range=(-1.0, 1.0))
print('WRAPPED ACTION SPACE: ', env.get_wrapper_attr('action_space'))

env.reset()
for i in range(5):
    action = env.action_space.sample()
    print('Normalized action: ', action)
    _, _, _, _, info = env.step(action)
    print('Action performed in the simulator: ', info['action'])
env.close()

#==============================================================================================#   
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.                               
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-continuous-v1                           
#==============================================================================================# 
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-continuous-v1-res1
[MODEL] (INFO) : Model Config is correct.                                            
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names. 
[MODEL] (INFO) : Updated building model Output:Meter with meter names.               
[MODEL] (INFO) : Runperiod established.                                              
[MODEL] (INFO) : Episode length (seconds): 31536000.0                                
[MODEL] (INFO) : timestep size (seconds): 900.0                                      
[MODE

## Action discretization wrapper

Let's see how to discretize a continuous action space. We will need to specify the **new discrete action space** and an **action mapping function** whose output matches the original unwrapped action space:

In [5]:
# We will create a continuous environment
env = gym.make('Eplus-5zone-hot-continuous-v1')
print('ORIGINAL ACTION SPACE: ', env.get_wrapper_attr('action_space'))
print('IS DISCRETE?: ', env.get_wrapper_attr('is_discrete'))

# Defining new discrete space and action mapping function
new_discrete_space = gym.spaces.Discrete(10)  # Action values [0,9]


def action_mapping_function(action):
    mapping = {
        0: np.array([12, 30], dtype=np.float32),
        1: np.array([13, 29], dtype=np.float32),
        2: np.array([14, 28], dtype=np.float32),
        3: np.array([15, 27], dtype=np.float32),
        4: np.array([16, 26], dtype=np.float32),
        5: np.array([17, 25], dtype=np.float32),
        6: np.array([18, 24], dtype=np.float32),
        7: np.array([19, 23.25], dtype=np.float32),
        8: np.array([20, 23.25], dtype=np.float32),
        9: np.array([21, 23.25], dtype=np.float32)
    }

    return mapping[action]


# Apply the discretization wrapper
env = DiscretizeEnv(env, discrete_space=new_discrete_space,
                    action_mapping=action_mapping_function)
print('WRAPPED ACTION SPACE: ', env.get_wrapper_attr('action_space'))
print('IS DISCRETE?: ', env.get_wrapper_attr('is_discrete'))
env.reset()
for i in range(5):
    action = env.action_space.sample()
    print('ACTION DISCRETE: ', action)
    _, _, _, _, info = env.step(action)
    print('Action done in simulator: ', info['action'])
env.close()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-continuous-v1
#==============================================================================================#
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-continuous-v1-res1
[MODEL] (INFO) : Model Config is correct.
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names.
[MODEL] (INFO) : Updated building model Output:Meter with meter names.
[MODEL] (INFO) : Runperiod established.
[MODEL] (INFO) : Episode length (seconds): 31536000.0
[MODEL] (INFO) : timestep size (seconds): 900.0
[MODEL] (INFO) : timesteps per episode: 35040
[REWARD] (INFO) : Reward function initialized.
[ENVIRONMENT] (INFO) : Environment created successfully.
ORIGINAL ACTION SPACE:  Box([12.   23.25], [23.25 30.  ], (2,), float32)
IS DISCRETE?:  False

## Discrete incremental wrapper

This wrapper updates an environment to utilize an incremental setpoint action space.It converts the environment into a *discrete* environment with an action mapping function and action space depending on the `step` and `delta` values specified. 

The action is added to the **current setpoint** value instead of overwriting the latest action. Thus, the action is the current setpoint values with the applied increment/decrement, rather than the discrete value action that defines the increment/decrement itself.

In [6]:
env = gym.make('Eplus-5zone-hot-continuous-v1')
print('ORIGINAL ACTION SPACE: ', env.get_wrapper_attr('action_space'))

env = DiscreteIncrementalWrapper(
    env, initial_values=[21.0, 25.0], delta_temp=2, step_temp=0.5)

print('WRAPPED ACTION SPACE: ', env.get_wrapper_attr('action_space'))
print('WRAPPED ACTION MAPPING: ', env.get_wrapper_attr('action_mapping'))

#==============================================================================================#   
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.                               
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-continuous-v1                           
#==============================================================================================# 
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-continuous-v1-res1
[MODEL] (INFO) : Model Config is correct.                                            
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names. 
[MODEL] (INFO) : Updated building model Output:Meter with meter names.               
[MODEL] (INFO) : Runperiod established.                                              
[MODEL] (INFO) : Episode length (seconds): 31536000.0                                
[MODEL] (INFO) : timestep size (seconds): 900.0                                      
[MODE

The maximum and minimum values defined when creating the action mapping are read from the environment action space, ensuring that the setpoint increments and decrements do not exceed the corresponding limits.

The `delta` and `step` values are used to determine how the discrete space of these increments and decrements will be constructed.

Here's an example of how it works:

In [7]:
env.reset()
print('CURRENT SETPOINTS VALUES: ', env.get_wrapper_attr('current_setpoints'))

for i in range(5):
    action = env.action_space.sample()
    _, _, _, _, info = env.step(action)
    print('Action number ', i, ': ',
          env.get_wrapper_attr('action_mapping')(action))
    print('Setpoints update: ', info['action'])
env.close()

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode.
[ENVIRONMENT] (INFO) : Episode 1: Eplus-5zone-hot-continuous-v1
#----------------------------------------------------------------------------------------------#
[MODEL] (INFO) : Episode directory created.
[MODEL] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODEL] (INFO) : Adapting weather to building model.
[ENVIRONMENT] (INFO) : Saving episode output path in /workspaces/sinergym/examples/Eplus-5zone-hot-continuous-v1-res1/episode-1/output.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
[ENVIRONMENT] (INFO) : Episode 1 started.
CURRENT SETPOINTS VALUES:  [21. 25.]
Action number  0 :  [0. 2.]
Setpoints update:  [21.0, 27.0]
Action number  1 :  [0. 1.]
Setpoints update:  [21.0, 28.0]
Action number  2 :  [0.  1.5]
Setpoints update:  [21.0, 29.5]
Ac

## Normalization wrapper

This wrapper is used to transform observations received from the simulator to values between in range `[-1, 1]`.

It is based on [Gymnasium's dynamic normalization wrapper](https://gymnasium.farama.org/_modules/gymnasium/wrappers/normalize/#NormalizeObservation). 

Until properly calibrated, it may not be precise, and the values may often be out of range, so use this wrapper with caution.

In [8]:
# Original env
env = gym.make('Eplus-5zone-hot-discrete-v1')

# Normalized env
env = NormalizeObservation(
    env=env)

#==============================================================================================#   
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.                               
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-discrete-v1                             
#==============================================================================================# 
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1
[MODEL] (INFO) : Model Config is correct.                                            
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names. 
[MODEL] (INFO) : Updated building model Output:Meter with meter names.               
[MODEL] (INFO) : Runperiod established.                                              
[MODEL] (INFO) : Episode length (seconds): 31536000.0                                
[MODEL] (INFO) : timestep size (seconds): 900.0                                      
[MODEL]

You can check how the specified variables have been correctly normalized:

In [ ]:
env.reset()

obs, _, _, _, _ = env.step(env.action_space.sample())
obs_dict = env.get_obs_dict(obs)
env.close()

print('OBSERVATION WITH NORMALIZATION: ', obs_dict)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode.
[ENVIRONMENT] (INFO) : Episode 1: Eplus-5zone-hot-discrete-v1
#----------------------------------------------------------------------------------------------#
[MODEL] (INFO) : Episode directory created.
[MODEL] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODEL] (INFO) : Adapting weather to building model.
[ENVIRONMENT] (INFO) : Saving episode output path in /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1/episode-1/output.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
[ENVIRONMENT] (INFO) : Episode 1 started.
[WRAPPER NormalizeObservation] (INFO) : Normalization calibration saved.
[ENVIRONMENT] (INFO) : Environment closed. [Eplus-5zone-hot-discrete-v1]                  
Simulation Progress [Episode 1]: 100%|██████████| 100/100 [00

## Logging and storing data with logger wrappers

### LoggerWrapper layer

This wrapper uses *Sinergym*'s `LoggerStorage` class to properly capture the interaction flow with the environment.

The class used by the wrapper can be replaced with a different back-end. It can then be combined with different wrappers to save the stored data, such as `CSVLogger` or `WandBLogger`. For more information about *Sinergym*'s logger, visit [Logging System Overview](https://ugr-sail.github.io/sinergym/compilation/main/pages/logging.html#logging-system-overview), [Logger Wrappers](https://ugr-sail.github.io/sinergym/compilation/main/pages/wrappers.html#logger-wrappers) and [an example about custom loggers](https://ugr-sail.github.io/sinergym/compilation/main/pages/notebooks/personalize_loggerwrapper.html).

In [10]:
env = gym.make('Eplus-5zone-hot-discrete-v1')
env = LoggerWrapper(env, storage_class=LoggerStorage)

#==============================================================================================#   
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.                               
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-discrete-v1                             
#==============================================================================================# 
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1
[MODEL] (INFO) : Model Config is correct.                                            
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names. 
[MODEL] (INFO) : Updated building model Output:Meter with meter names.               
[MODEL] (INFO) : Runperiod established.                                              
[MODEL] (INFO) : Episode length (seconds): 31536000.0                                
[MODEL] (INFO) : timestep size (seconds): 900.0                                      
[MODEL]

This wrapper enables the use of a `LoggerStorage` instance within the environment class and automatically captures interaction data while actions are sent by an agent. At each reset, the data from this class is cleared to start the next episode. The idea is to combine it with other output loggers like those listed below:

### LoggerCSV layer

In [11]:
env = CSVLogger(env)

env.reset()
truncated = terminated = False
current_month = 0

while not (terminated or truncated):
    a = env.action_space.sample()
    _, _, terminated, truncated, _ = env.step(a)
env.close()

[WRAPPER CSVLogger] (INFO) : Wrapper initialized.
#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode.
[ENVIRONMENT] (INFO) : Episode 1: Eplus-5zone-hot-discrete-v1
#----------------------------------------------------------------------------------------------#
[MODEL] (INFO) : Episode directory created.
[MODEL] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODEL] (INFO) : Adapting weather to building model.
[ENVIRONMENT] (INFO) : Saving episode output path in /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1/episode-1/output.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
[ENVIRONMENT] (INFO) : Episode 1 started.
[WRAPPER CSVLogger] (INFO) : Environment closed, data updated in monitor and progress.csv.
Simulation Progress [Episode 1]: 100%|██████████| 100/100 [00:23<00:00,  4.34%/s, 10

Once the `LoggerWrapper` is applied, this wrapper can be used to output episode data through *Sinergym*’s output, along with summary metrics added to CSV files. More details on this structure can be found in [OutputFormat](https://ugr-sail.github.io/sinergym/compilation/main/pages/output.html). 

*Sinergym* will raise an error if this wrapper is used without first enabling `LoggerWrapper` or a similar custom logger.

### WandBLogger layer

In [ ]:
# env = WandBLogger(env = Env,
#                 entity = <wandb_account_entity>,
#                 project_name = <wandb_project_name>,
#                 run_name = <run_name>
#                 group = 'Notebook_example',
#                 tags: ['tag1','tag2'],
#                 save_code = False,
#                 dump_frequency = 1000,
#                 artifact_save = True,
#                 artifact_type = 'output',
#                 excluded_info_keys = ['reward',
#                                   'action',
#                                   'timestep',
#                                   'month',
#                                   'day',
#                                   'hour',
#                                   'time_elapsed(hours)',
#                                   'reward_weight',
#                                   'is_raining'],
#                 excluded_episode_summary_keys = ['terminated',
#                                              'truncated']):

# env.reset()
# truncated = terminated = False
# current_month = 0
# while not (terminated or truncated):
#     a = env.action_space.sample()
#     _,_,terminated,truncated,_=env.step(a)
# env.close()

Similar to `CSVLogger`, this wrapper requires the environment to have been previously encapsulated by a `LoggerWrapper` or any custom logger.

The user must have a pre-existing **Weights and Biases** account and correctly configured it. 

This wrapper does not override `CSVLogger`, so both can be applied simultaneously.

## Multi-observation wrapper

This wrapper stacks observations in a history queue, whose size can be customized:

In [ ]:
# Original environment
env = gym.make('Eplus-5zone-hot-discrete-v1')
obs, info = env.reset()
print('BEFORE MULTI OBSERVATION: ', obs)

# Multi-observation environment with a queue of size 5
env = MultiObsWrapper(env, n=5, flatten=True)
obs, info = env.reset()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-discrete-v1
#==============================================================================================#
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1
[MODEL] (INFO) : Model Config is correct.
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names.
[MODEL] (INFO) : Updated building model Output:Meter with meter names.
[MODEL] (INFO) : Runperiod established.
[MODEL] (INFO) : Episode length (seconds): 31536000.0
[MODEL] (INFO) : timestep size (seconds): 900.0
[MODEL] (INFO) : timesteps per episode: 35040
[REWARD] (INFO) : Reward function initialized.
[ENVIRONMENT] (INFO) : Environment created successfully.
[WRAPPER DiscretizeEnv] (INFO) : New Discrete Space and mapping: Discrete(10)
[WRAPPER Discretize

Simulation Progress [Episode 2]: 100%|██████████| 100/100 [00:00<00:00, 194180.74%/s, 100% completed] 

The result is:

In [13]:
print('MULTI OBSERVATION: \n', obs)
env.close()

MULTI OBSERVATION: 
 [1.00000000e+00 1.00000000e+00 0.00000000e+00 4.40000010e+00
 6.50000000e+01 3.87500000e+00 1.45000000e+02 0.00000000e+00
 0.00000000e+00 1.28000002e+01 4.00000000e+01 1.99520779e+01
 2.82156696e+01 0.00000000e+00 0.00000000e+00 1.17947556e+02
 1.06152805e+05 1.00000000e+00 1.00000000e+00 0.00000000e+00
 4.40000010e+00 6.50000000e+01 3.87500000e+00 1.45000000e+02
 0.00000000e+00 0.00000000e+00 1.28000002e+01 4.00000000e+01
 1.99520779e+01 2.82156696e+01 0.00000000e+00 0.00000000e+00
 1.17947556e+02 1.06152805e+05 1.00000000e+00 1.00000000e+00
 0.00000000e+00 4.40000010e+00 6.50000000e+01 3.87500000e+00
 1.45000000e+02 0.00000000e+00 0.00000000e+00 1.28000002e+01
 4.00000000e+01 1.99520779e+01 2.82156696e+01 0.00000000e+00
 0.00000000e+00 1.17947556e+02 1.06152805e+05 1.00000000e+00
 1.00000000e+00 0.00000000e+00 4.40000010e+00 6.50000000e+01
 3.87500000e+00 1.45000000e+02 0.00000000e+00 0.00000000e+00
 1.28000002e+01 4.00000000e+01 1.99520779e+01 2.82156696e+01
 0.

## Weather forecasting wrapper

This wrapper adds weather forecast information to the current observation.

In [14]:
# Original environment
env = gym.make('Eplus-5zone-hot-discrete-v1')
obs, info = env.reset()
print('OBSERVATION VARIABLES BEFORE WEATHER FORECASTING: ',
      env.get_wrapper_attr('observation_variables'))
print('OBSERVATION BEFORE WEATHER FORECASTING: ', obs)

# Weather forecasting environment
env = WeatherForecastingWrapper(env, n=5, delta=1)
obs, info = env.reset()

#==============================================================================================#   
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.                               
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-discrete-v1                             
#==============================================================================================# 
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1
[MODEL] (INFO) : Model Config is correct.                                            
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names. 
[MODEL] (INFO) : Updated building model Output:Meter with meter names.               
[MODEL] (INFO) : Runperiod established.                                              
[MODEL] (INFO) : Episode length (seconds): 31536000.0                                
[MODEL] (INFO) : timestep size (seconds): 900.0                                      
[MODEL]

We can observe the results:

In [15]:
print('OBSERVATION VARIABLES AFTER WEATHER FORECASTING: ',
      env.get_wrapper_attr('observation_variables'))

print('OBSERVATION AFTER WEATHER FORECASTING: ', obs)

OBSERVATION VARIABLES AFTER WEATHER FORECASTING:  ['month', 'day_of_month', 'hour', 'outdoor_temperature', 'outdoor_humidity', 'wind_speed', 'wind_direction', 'diffuse_solar_radiation', 'direct_solar_radiation', 'htg_setpoint', 'clg_setpoint', 'air_temperature', 'air_humidity', 'people_occupant', 'co2_emission', 'HVAC_electricity_demand_rate', 'total_electricity_HVAC', 'forecast_1_Dry Bulb Temperature', 'forecast_1_Relative Humidity', 'forecast_1_Wind Direction', 'forecast_1_Wind Speed', 'forecast_1_Direct Normal Radiation', 'forecast_1_Diffuse Horizontal Radiation', 'forecast_2_Dry Bulb Temperature', 'forecast_2_Relative Humidity', 'forecast_2_Wind Direction', 'forecast_2_Wind Speed', 'forecast_2_Direct Normal Radiation', 'forecast_2_Diffuse Horizontal Radiation', 'forecast_3_Dry Bulb Temperature', 'forecast_3_Relative Humidity', 'forecast_3_Wind Direction', 'forecast_3_Wind Speed', 'forecast_3_Direct Normal Radiation', 'forecast_3_Diffuse Horizontal Radiation', 'forecast_4_Dry Bulb T

## Energy cost wrapper

This wrapper adds energy cost information to the current observation:

In [16]:
# Original environment
env = gym.make('Eplus-5zone-hot-discrete-v1')
obs, info = env.reset()
print('OBSERVATION VARIABLES BEFORE ADDING ENERGY COST: \n',
      env.get_wrapper_attr('observation_variables'))
print('OBSERVATION VALUES BEFORE ADDING ENERGY COST: \n', obs)

# Energy Cost environment
env = EnergyCostWrapper(
    env, energy_cost_data_path='/workspaces/sinergym/sinergym/data/energy_cost/PVPC_active_energy_billing_Iberian_Peninsula_2023.csv')
obs, info = env.reset()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-discrete-v1
#==============================================================================================#
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-discrete-v1-res1
[MODEL] (INFO) : Model Config is correct.
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names.
[MODEL] (INFO) : Updated building model Output:Meter with meter names.
[MODEL] (INFO) : Runperiod established.
[MODEL] (INFO) : Episode length (seconds): 31536000.0
[MODEL] (INFO) : timestep size (seconds): 900.0
[MODEL] (INFO) : timesteps per episode: 35040
[REWARD] (INFO) : Reward function initialized.
[ENVIRONMENT] (INFO) : Environment created successfully.
[WRAPPER DiscretizeEnv] (INFO) : New Discrete Space and mapping: Discrete(10)
[WRAPPER Discretize

This is the result:

In [17]:
print('OBSERVATION VARIABLES AFTER ADDING ENERGY COST: \n', env.get_wrapper_attr('observation_variables'))
print('OBSERVATION VALUES AFTER ADDING ENERGY COST: \n',obs)

OBSERVATION VARIABLES AFTER ADDING ENERGY COST: 
 ['month', 'day_of_month', 'hour', 'outdoor_temperature', 'outdoor_humidity', 'wind_speed', 'wind_direction', 'diffuse_solar_radiation', 'direct_solar_radiation', 'htg_setpoint', 'clg_setpoint', 'air_temperature', 'air_humidity', 'people_occupant', 'co2_emission', 'HVAC_electricity_demand_rate', 'total_electricity_HVAC', 'energy_cost']
OBSERVATION VALUES AFTER ADDING ENERGY COST: 
 [1.00000000e+00 1.00000000e+00 0.00000000e+00 4.40000010e+00
 6.50000000e+01 3.87500000e+00 1.45000000e+02 0.00000000e+00
 0.00000000e+00 1.28000002e+01 4.00000000e+01 1.99520779e+01
 2.82156696e+01 0.00000000e+00 0.00000000e+00 1.17947556e+02
 1.06152805e+05 4.14500000e+01]


## Nesting wrappers

All wrappers included in *Sinergym* are stackable and organized in layers. However, the order in which these layers are applied can affect the final result, depending on the wrappers being used.

For instance, applying the logger before normalizing differs from doing it in the reverse order. In the first case, the data will be logged without normalization, even though the agent will operate in a normalized environment. In the second case, the logger will capture the normalized values since it encapsulates the normalization applied by the previous layer.

An example of how to nest wrappers is shown below:

In [18]:
env = gym.make('Eplus-5zone-hot-continuous-v1')
env = MultiObjectiveReward(
    env=env,
    reward_terms=[
        'energy_term',
        'comfort_term'])
env = PreviousObservationWrapper(env, previous_variables=[
    'htg_setpoint',
    'clg_setpoint',
    'air_temperature'])
env = DatetimeWrapper(env)
env = DiscreteIncrementalWrapper(
    env, initial_values=[21.0, 25.0], delta_temp=2, step_temp=0.5)
env = NormalizeObservation(
    env=env)
env = LoggerWrapper(env=env)
env = MultiObsWrapper(env=env, n=5, flatten=True)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.
[ENVIRONMENT] (INFO) : Name: Eplus-5zone-hot-continuous-v1
#==============================================================================================#
[MODEL] (INFO) : Working directory created: /workspaces/sinergym/examples/Eplus-5zone-hot-continuous-v1-res1
[MODEL] (INFO) : Model Config is correct.
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names.
[MODEL] (INFO) : Updated building model Output:Meter with meter names.
[MODEL] (INFO) : Runperiod established.
[MODEL] (INFO) : Episode length (seconds): 31536000.0
[MODEL] (INFO) : timestep size (seconds): 900.0
[MODEL] (INFO) : timesteps per episode: 35040
[REWARD] (INFO) : Reward function initialized.
[ENVIRONMENT] (INFO) : Environment created successfully.
[WRAPPER MultiObjectiveReward] (INFO) : wrapper initialized.
[WRAPPER PreviousObservationWrap

Now we can simply use the wrapped environment as follows:

In [19]:
for i in range(1):
    obs, info = env.reset()
    truncated = terminated = False
    current_month = 0
    while not (terminated or truncated):
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', reward, info)
env.close()

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode.
[ENVIRONMENT] (INFO) : Episode 1: Eplus-5zone-hot-continuous-v1
#----------------------------------------------------------------------------------------------#
[MODEL] (INFO) : Episode directory created.
[MODEL] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODEL] (INFO) : Adapting weather to building model.
[ENVIRONMENT] (INFO) : Saving episode output path in /workspaces/sinergym/examples/Eplus-5zone-hot-continuous-v1-res1/episode-1/output.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
[ENVIRONMENT] (INFO) : Episode 1 started.
Reward:  [-0.48383429007956374, 0.0] {'time_elapsed(hours)': 0.5416666666666666, 'month': 1, 'day': 1, 'hour': 0, 'is_raining': False, 'action': [21.0, 23.25], 'timestep': 1, 'reward': -0.48383429007956374, 'energy_term':